In [ ]:
# !pip install tensorflow --quiet

In [20]:
# importing
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Bidirectional, Dropout, Dense, LSTM
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from utils import load_nusax_data
from model import SimpleRNNModel
from layers import RNN as CustomRNN, BiRNN as CustomBiRNN

In [21]:
# ----- Fixed Hyperparameters -----
MAX_TOKENS    = 20000
SEQ_LENGTH    = 100
EMBED_DIM     = 128
DROPOUT_RATE  = 0.5
BATCH_SIZE    = 50
EPOCHS        = 15

In [22]:
# ----- Load and preprocess data -----
tok_train, y_train, tok_val, y_val, tok_test, y_test, vocab_size, num_classes = \
    load_nusax_data(MAX_TOKENS, SEQ_LENGTH)

os.makedirs('results/weights', exist_ok=True)
os.makedirs('results/plots', exist_ok=True)
results_summary = []

In [23]:
# ----- Needed Function Definitions -----
def build_keras_model(num_rnn_layers, rnn_units, bidirectional, rnn_type='SimpleRNN', dropout_rate=DROPOUT_RATE):
    keras_model = Sequential()
    keras_model.add(Embedding(
        input_dim=vocab_size,
        output_dim=EMBED_DIM,
        input_length=SEQ_LENGTH,
        name='embedding'
    ))

    for i in range(num_rnn_layers):
        is_last_rnn = (i == num_rnn_layers - 1)
        return_sequences = not is_last_rnn
        
        layer_name_prefix = f"{'bi' if bidirectional else ''}{rnn_type.lower()}_{i}"

        if rnn_type == 'SimpleRNN':
            rnn_layer_constructor = SimpleRNN(rnn_units, return_sequences=return_sequences, name=f"{layer_name_prefix}_simplernn")
        elif rnn_type == 'LSTM':
            rnn_layer_constructor = LSTM(rnn_units, return_sequences=return_sequences, name=f"{layer_name_prefix}_lstm")
        else:
            raise ValueError(f"Unsupported RNN type: {rnn_type}")

        if bidirectional:
            keras_model.add(Bidirectional(rnn_layer_constructor, name=f"{layer_name_prefix}_bidir"))
        else:
            keras_model.add(rnn_layer_constructor)
            
    keras_model.add(Dropout(dropout_rate, name='dropout'))
    keras_model.add(Dense(num_classes, activation='softmax', name='output'))

    keras_model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return keras_model

def plot_history(history, model_name, experiment_name):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Loss - {model_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Accuracy - {model_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plot_filename = f"results/plots/{experiment_name}_{model_name}_history.png"
    plt.savefig(plot_filename)
    print(f"Saved plot to {plot_filename}")
    plt.close()

In [24]:
# --- Experiment Configurations ---
experiments = []

# 1. beda layer RNN
base_rnn_units = 32
base_bidirectional = False
for num_layers in [5, 6, 7]:
    experiments.append({
        "name": f"NumLayers_{num_layers}_Units_{base_rnn_units}_Dir_{'Bi' if base_bidirectional else 'Uni'}",
        "num_rnn_layers": num_layers,
        "rnn_units": base_rnn_units,
        "bidirectional": base_bidirectional,
        "compare_custom": (num_layers == 5 and not base_bidirectional)
    })

# 2. beda cell RNN per layer
base_num_layers = 5
base_bidirectional = False
for rnn_units in [32, 64, 128]:
    experiments.append({
        "name": f"NumLayers_{base_num_layers}_Units_{rnn_units}_Dir_{'Bi' if base_bidirectional else 'Uni'}",
        "num_rnn_layers": base_num_layers,
        "rnn_units": rnn_units,
        "bidirectional": base_bidirectional,
        "compare_custom": (base_num_layers == 5 and not base_bidirectional and rnn_units==32)
    })

# 3. beda RNN berdasarkan arah
base_num_layers = 5
base_rnn_units = 32
for bidirectional_flag in [False, True]:
    experiments.append({
        "name": f"NumLayers_{base_num_layers}_Units_{base_rnn_units}_Dir_{'Bi' if bidirectional_flag else 'Uni'}",
        "num_rnn_layers": base_num_layers,
        "rnn_units": base_rnn_units,
        "bidirectional": bidirectional_flag,
        "compare_custom": (base_num_layers == 5 and base_rnn_units==32)
    })

In [ ]:
# --- Run Experiments ---
for i, exp_config in enumerate(experiments):
    print(f"\n--- Running Experiment {i+1}/{len(experiments)}: {exp_config['name']} ---")
    
    model_name = exp_config['name']
    num_rnn_layers_exp = exp_config['num_rnn_layers']
    rnn_units_exp = exp_config['rnn_units']
    bidirectional_exp = exp_config['bidirectional']

    # ----- bikin model keras -----
    keras_model = build_keras_model(
        num_rnn_layers=num_rnn_layers_exp,
        rnn_units=rnn_units_exp,
        bidirectional=bidirectional_exp
    )
    keras_model.summary()

    # ----- Training -----
    print(f"Training Keras model: {model_name}")
    history = keras_model.fit(
        tok_train, y_train,
        validation_data=(tok_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=2
    )
    # Save weights
    weights_path = f'results/weights/keras_rnn_{model_name}.weights.h5'
    keras_model.save_weights(weights_path)
    print(f"Saved Keras weights to {weights_path}")

    # Plot history
    plot_history(history, model_name, "experiment")

    # ----- Evaluation on test set (Keras) -----
    y_pred_prob_keras = keras_model.predict(tok_test, batch_size=BATCH_SIZE)
    y_pred_keras = np.argmax(y_pred_prob_keras, axis=1)
    f1_test_keras = f1_score(y_test, y_pred_keras, average='macro')
    print(f"[Keras - {model_name}] Test Macro F1-score: {f1_test_keras:.4f}")
    
    results_summary.append({
        "name": model_name,
        "num_rnn_layers": num_rnn_layers_exp,
        "rnn_units": rnn_units_exp,
        "bidirectional": bidirectional_exp,
        "f1_keras": f1_test_keras,
        "f1_custom": None
    })


--- Running Experiment 1/8: NumLayers_5_Units_32_Dir_Uni ---


c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simplernn_0_simplernn           │ ?                      │   0 (unbuilt) │
│ (SimpleRNN)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simplernn_1_simplernn           │ ?                      │   0 (unbuilt) │
│ (SimpleRNN)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simplernn_2_simplernn           │ ?                      │   0 (unbuilt) │
│ (SimpleRNN)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simplernn_3_simplernn           │ ?                      │   0 (unbuilt) │
│ (SimpleRNN)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simplernn_4_simplernn           │ ?                      │   0 (unbuilt) │
│ (SimpleRNN)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bisimplernn_0_bidir             │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bisimplernn_1_bidir             │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bisimplernn_2_bidir             │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bisimplernn_3_bidir             │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bisimplernn_4_bidir             │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Training Keras model: NumLayers_5_Units_32_Dir_Bi
Epoch 1/15
10/10 - 9s - 933ms/step - accuracy: 0.3640 - loss: 1.2446 - val_accuracy: 0.4300 - val_loss: 1.0726
Epoch 2/15
10/10 - 1s - 86ms/step - accuracy: 0.5420 - loss: 0.9471 - val_accuracy: 0.4400 - val_loss: 1.0191
Epoch 3/15
10/10 - 1s - 81ms/step - accuracy: 0.7340 - loss: 0.6411 - val_accuracy: 0.5700 - val_loss: 0.9634
Epoch 4/15
10/10 - 1s - 82ms/step - accuracy: 0.8760 - loss: 0.3940 - val_accuracy: 0.5800 - val_loss: 0.9829
Epoch 5/15
10/10 - 1s - 73ms/step - accuracy: 0.9480 - loss: 0.2254 - val_accuracy: 0.5400 - val_loss: 1.0476
Epoch 6/15
10/10 - 1s - 74ms/step - accuracy: 0.9800 - loss: 0.1261 - val_accuracy: 0.6100 - val_loss: 1.0536
Epoch 7/15
10/10 - 1s - 71ms/step - accuracy: 0.9980 - loss: 0.0680 - val_accuracy: 0.6100 - val_loss: 1.0809
Epoch 8/15
10/10 - 1s - 76ms/step - accuracy: 0.9980 - loss: 0.0514 - val_accuracy: 0.5800 - val_loss: 1.0871
Epoch 9/15
10/10 - 1s - 81ms/step - accuracy: 0.9980 - loss: 0.0428 -

In [26]:
# ----- Custom Model Comparison -----
if exp_config.get("compare_custom", False):
    print(f"\n--- Comparing with Custom Model for {model_name} ---")
    
    custom_return_sequences = [True] * (num_rnn_layers_exp - 1) + [False] if num_rnn_layers_exp > 0 else []

    custom_model = SimpleRNNModel(
        vocab_size=vocab_size,
        embed_dim=EMBED_DIM,
        hidden_dim=rnn_units_exp,
        num_classes=num_classes,
        dropout_rate=DROPOUT_RATE,
        num_layers=num_rnn_layers_exp,
        bidirectional=bidirectional_exp,
        return_sequences_list=custom_return_sequences
    )

    keras_weights_list = keras_model.get_weights()
    custom_model.load_keras_weights(keras_weights_list)
    print("Loaded Keras weights into custom model.")

    custom_model.set_training_mode(False)

    probs_custom = custom_model.forward(tok_test, training=False)
    preds_custom = np.argmax(probs_custom, axis=1)
    f1_custom = f1_score(y_test, preds_custom, average='macro')
    print(f"[Custom - {model_name}] Test Macro F1-score: {f1_custom:.4f}")
    
    for res in results_summary:
        if res["name"] == model_name:
            res["f1_custom"] = f1_custom
            break
    
    if not np.isclose(f1_test_keras, f1_custom, atol=1e-3):
        print("Warning: Keras and Custom F1 scores differ significantly. Debugging may be needed.")


--- Comparing with Custom Model for NumLayers_5_Units_32_Dir_Bi ---
Loaded Keras weights into custom model.
[Custom - NumLayers_5_Units_32_Dir_Bi] Test Macro F1-score: 0.4896


In [27]:
# --- Print Hasil ---
print("\n\n--- Experiment Results Summary ---")
print("Name | RNN Layers | RNN Units | Bidirectional | F1 Keras | F1 Custom")
print("-" * 80)
for res in results_summary:
    f1_custom_str = f"{res['f1_custom']:.4f}" if res['f1_custom'] is not None else "N/A"
    print(f"{res['name']} | {res['num_rnn_layers']} | {res['rnn_units']} | {res['bidirectional']} | {res['f1_keras']:.4f} | {f1_custom_str}")



--- Experiment Results Summary ---
Name | RNN Layers | RNN Units | Bidirectional | F1 Keras | F1 Custom
--------------------------------------------------------------------------------
NumLayers_5_Units_32_Dir_Uni | 5 | 32 | False | 0.3277 | N/A
NumLayers_6_Units_32_Dir_Uni | 6 | 32 | False | 0.3390 | N/A
NumLayers_7_Units_32_Dir_Uni | 7 | 32 | False | 0.3523 | N/A
NumLayers_5_Units_32_Dir_Uni | 5 | 32 | False | 0.2838 | N/A
NumLayers_5_Units_64_Dir_Uni | 5 | 64 | False | 0.1844 | N/A
NumLayers_5_Units_128_Dir_Uni | 5 | 128 | False | 0.1827 | N/A
NumLayers_5_Units_32_Dir_Uni | 5 | 32 | False | 0.3500 | N/A
NumLayers_5_Units_32_Dir_Bi | 5 | 32 | True | 0.4896 | 0.4896
